# Simple CNN notebook

## Import Libraries

In [66]:
%matplotlib inline
import numpy as np
from utils.helper import read_data, blur, to_real, morphology
import cv2
import matplotlib.pyplot as plt
from tensorflow import keras
FOLDER = "data/data_big/"

## Load data

In [67]:
data_empty = read_data(FOLDER + "empty.csv")
data_1p = read_data(FOLDER + "1p.csv")
data_2p = read_data(FOLDER + "2p.csv")
data_3p = read_data(FOLDER + "3p.csv")
print("----------------Shapes----------------")
print("empty:", data_empty.shape)
print("1p:", data_1p.shape)
print("2p:", data_2p.shape)
print("3p:", data_3p.shape)

----------------Shapes----------------
empty: (480, 5, 3, 64, 64)
1p: (480, 5, 3, 64, 64)
2p: (480, 5, 3, 64, 64)
3p: (480, 5, 3, 64, 64)


In [68]:
LENGTH_EMPTY = data_empty.shape[0]
LENGTH_1P = data_1p.shape[0]
LENGTH_2P = data_2p.shape[0]
LENGTH_3P = data_3p.shape[0]

In [69]:
data_empty_summed = to_real(np.sum(data_empty, axis=1)) 
data_1p_summed = to_real(np.sum(data_1p, axis=1))
data_2p_summed = to_real(np.sum(data_2p, axis=1))
data_3p_summed = to_real(np.sum(data_3p, axis=1))

/Users/anatolii/Desktop/RWTH/eestech/hackathon-milan/submission/code/utils/helper.py:41: ComplexWarning: Casting complex values to real discards the imaginary part
  return data_real.astype(float)


In [70]:
data_0 = np.moveaxis(data_empty_summed, 1,3)
data_1 = np.moveaxis(data_1p_summed, 1,3)
data_2 = np.moveaxis(data_2p_summed, 1,3)
data_3 = np.moveaxis(data_3p_summed, 1,3)
print(data_0.shape)
print(data_1.shape)
print(data_2.shape)
print(data_3.shape)

(480, 64, 64, 3)
(480, 64, 64, 3)
(480, 64, 64, 3)
(480, 64, 64, 3)


In [143]:
data_tot = np.concatenate((data_0, data_1, data_2, data_3), axis=0)
data_tot.shape

(1920, 64, 64, 3)

In [127]:
data_tot[:,32,:,:] = 0
# plt.imshow(np.abs(data_tot[100,:,:,1]))
# plt.colorbar()
# plt.show()
# sum_lines = np.sum(data_tot[1000,:,:,1], axis=1)

# print(f"Noise velosity sample: {np.argmax(sum_lines)}")
# plt.plot(sum_lines)
# sum_lines.shape

In [176]:
data_tot_diff = np.diff(data_tot, axis = 0)
print(data_tot_diff.shape)
# data_tot_diff.shape
print(np.expand_dims(data_tot_diff[1918,:,:,:], axis = 0).shape)
new_data = np.concatenate((data_tot_diff, np.expand_dims(data_tot_diff[1918,:,:,:], axis = 0)), axis=0)
new_data.shape
# plt.imshow(np.abs(data_tot_diff[600,:,:,1]))

(1919, 64, 64, 3)
(1, 64, 64, 3)


(1919, 64, 64, 3)

### Create Labels

In [114]:
labels_empty = np.zeros(LENGTH_EMPTY)
labels_1p = np.ones(LENGTH_1P)
labels_2p = np.ones(LENGTH_2P) * 2
labels_3p = np.ones(LENGTH_3P) * 3
labels = np.concatenate((labels_empty, labels_1p, labels_2p, labels_3p), axis=0)
print("Labels shape:", labels.shape)

Labels shape: (1920,)


In [115]:
train_labels_one_hot = np.eye(4)[labels.astype(int)]
print("Labels abels one hot shape:", train_labels_one_hot.shape)

Labels abels one hot shape: (1920, 4)


In [149]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data_tot_diff, train_labels_one_hot, test_size=0.2)
print(f"Shape train_X: {train_X.shape}")
print(f"Shape test_X: {test_X.shape}")
print(f"Shape train_y: {train_y.shape}")
print(f"Shape test_y: {test_y.shape}")

ValueError: Found input variables with inconsistent numbers of samples: [1919, 1920]

In [ ]:
models = keras.models
layers = keras.layers

model = models.Sequential(
    [
        layers.Conv2D(8,
                      input_shape = (64,64,3),
                      kernel_size = 5,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                            strides = None,
                            padding = "valid"),
        layers.Conv2D(8,
                      input_shape = (32,32,8),
                      kernel_size = 3,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                    strides = None,
                    padding = "valid"),
        layers.Flatten(),
        layers.Dense(4),
        layers.Activation("softmax")
    ]
)
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 64, 64, 8)         608       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 32, 32, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 32, 32, 8)         584       
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 16, 16, 8)        0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 2048)              0         
                                                                 
 dense_8 (Dense)             (None, 4)                

In [ ]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

model.fit(
    train_X, train_y,
    epochs  = 10,
    verbose = 1,
    validation_split = 0.2,
    shuffle= True,
    callbacks=keras.callbacks.EarlyStopping(patience = 10)
)

model.save('models/CNN')

Epoch 1/10
39/39 [==============================] - 1s 20ms/step - loss: 1.4596 - accuracy: 0.5570 - val_loss: 0.5445 - val_accuracy: 0.7987
Epoch 2/10
39/39 [==============================] - 1s 19ms/step - loss: 0.4372 - accuracy: 0.8453 - val_loss: 0.3452 - val_accuracy: 0.9383
Epoch 3/10
39/39 [==============================] - 1s 18ms/step - loss: 0.2759 - accuracy: 0.9243 - val_loss: 0.2423 - val_accuracy: 0.9481
Epoch 4/10
39/39 [==============================] - 1s 19ms/step - loss: 0.2299 - accuracy: 0.9292 - val_loss: 0.2370 - val_accuracy: 0.8929
Epoch 5/10
39/39 [==============================] - 1s 18ms/step - loss: 0.1911 - accuracy: 0.9373 - val_loss: 0.1653 - val_accuracy: 0.9448
Epoch 6/10
39/39 [==============================] - 1s 18ms/step - loss: 0.1095 - accuracy: 0.9764 - val_loss: 0.1321 - val_accuracy: 0.9643
Epoch 7/10
39/39 [==============================] - 1s 18ms/step - loss: 0.0879 - accuracy: 0.9788 - val_loss: 0.1011 - val_accuracy: 0.9708
Epoch 8/10
39

INFO:tensorflow:Assets written to: models/CNN/assets


INFO:tensorflow:Assets written to: models/CNN/assets


In [ ]:
loss, aacuracy = model.evaluate(test_X, test_y)
print(f"Loss: {loss}")
print(f"Accuracy: {aacuracy}")

12/12 [==============================] - 0s 7ms/step - loss: 0.0600 - accuracy: 0.9844
Loss: 0.060046374797821045
Accuracy: 0.984375
